<a href="https://colab.research.google.com/github/Antonio24ch/Proyecto_Cambio_Divisas/blob/main/corregido%20-%20Cargar_los_datos_en_Amazon_Redshift_Entregable2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Cambio de Divisas

In [ ]:
!pip install sqlalchemy-redshift

In [ ]:
#Cargando los paquetes
#Requests nos proporciona las capacidades para enviar una solicitud HTTP a un servidor.
import requests
import yaml
import json
import pytz
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
import psycopg2

In [ ]:
with open('apikey.yaml', 'r') as file:
    api_keys = yaml.safe_load(file)
api_key = api_keys['api_key']

### Obteniendo el cierre - código que se ejecutará diario

In [ ]:
# Extraer la fecha actual para el horario de Mx
timezone = pytz.timezone('America/Mexico_City')
current_time = datetime.now(timezone)
formatted_date = current_time.strftime("%Y-%m-%d")

currencies_list = ['EUR', 'GBP', 'USD', 'PEN', 'BTC', 'KRW', 'INR', 'CNY', 'BRL', 'ARS', 'JPY']
currencies = ','.join(currencies_list)

base_url = f"https://api.apilayer.com/currency_data/timeframe?start_date={formatted_date}&end_date={formatted_date}&source=MXN&currencies={currencies}"

payload = {}
headers= {
  "apikey": api_key
}


response = requests.get(base_url, headers=headers, data=payload)
response.status_code

200

In [ ]:
print(json.dumps(response.json(), indent=4))

{
    "success": true,
    "timeframe": true,
    "start_date": "2024-01-02",
    "end_date": "2024-01-02",
    "source": "MXN",
    "quotes": {
        "2024-01-02": {
            "MXNEUR": 0.053654,
            "MXNGBP": 0.046508,
            "MXNUSD": 0.058705,
            "MXNPEN": 0.218,
            "MXNBTC": 1.305554e-06,
            "MXNKRW": 76.835704,
            "MXNINR": 4.887596,
            "MXNCNY": 0.41678,
            "MXNBRL": 0.289061,
            "MXNARS": 47.58379,
            "MXNJPY": 8.345262
        }
    }
}


In [ ]:
datos = json.loads(response.text)

Este dataframe trae únicamente una fila, la cual es el valor actual del día en curso que se estará almacenando en la tabla

In [ ]:
df = pd.DataFrame(datos)
df

,success,timeframe,start_date,end_date,source,quotes
2024-01-02,True,True,2024-01-02,2024-01-02,MXN,"{'MXNEUR': 0.053654, 'MXNGBP': 0.046508, 'MXNU..."


### Limpiando datos

In [ ]:
df_quotes = pd.json_normalize(df['quotes'])


# Combinar las nuevas columnas con el DataFrame original
df = pd.concat([df, df_quotes], axis=1).drop('quotes', axis=1)

df

,success,timeframe,start_date,end_date,source,MXNEUR,MXNGBP,MXNUSD,MXNPEN,MXNBTC,MXNKRW,MXNINR,MXNCNY,MXNBRL,MXNARS,MXNJPY
2024-01-02,True,True,2024-01-02,2024-01-02,MXN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,0.053654,0.046508,0.058705,0.218,0.000001,76.835704,4.887596,0.41678,0.289061,47.58379,8.345262


In [ ]:
combined_row = df.iloc[0].combine_first(df.iloc[1])
# Convertir la serie combinada en un DataFrame de una sola fila
df_combined = pd.DataFrame([combined_row])
df_combined.rename(columns=lambda x: x.replace('MXN', '') if 'MXN' in x else x, inplace=True)
columns_to_drop = ['success', 'timeframe']
df_clean = df_combined.drop(columns=columns_to_drop).reset_index(drop=True)
df_clean

,start_date,end_date,source,EUR,GBP,USD,PEN,BTC,KRW,INR,CNY,BRL,ARS,JPY
0,2024-01-02,2024-01-02,MXN,0.053654,0.046508,0.058705,0.218,0.000001,76.835704,4.887596,0.41678,0.289061,47.58379,8.345262


In [ ]:
# Conocer el tipo de dato en el df
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   start_date  1 non-null      object 
 1   end_date    1 non-null      object 
 2   source      1 non-null      object 
 3   EUR         1 non-null      float64
 4   GBP         1 non-null      float64
 5   USD         1 non-null      float64
 6   PEN         1 non-null      float64
 7   BTC         1 non-null      float64
 8   KRW         1 non-null      float64
 9   INR         1 non-null      float64
 10  CNY         1 non-null      float64
 11  BRL         1 non-null      float64
 12  ARS         1 non-null      float64
 13  JPY         1 non-null      float64
dtypes: float64(11), object(3)
memory usage: 240.0+ bytes


### Cambiando tipo de datos

In [ ]:
# Como se esta trabajando con fechas, es importante cambiar el tipo de dato de object a fecha
columnas = ['start_date', 'end_date']
df_clean[columnas] = df_clean[columnas].apply(pd.to_datetime, errors='coerce', axis=1)
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_date  1 non-null      datetime64[ns]
 1   end_date    1 non-null      datetime64[ns]
 2   source      1 non-null      object        
 3   EUR         1 non-null      float64       
 4   GBP         1 non-null      float64       
 5   USD         1 non-null      float64       
 6   PEN         1 non-null      float64       
 7   BTC         1 non-null      float64       
 8   KRW         1 non-null      float64       
 9   INR         1 non-null      float64       
 10  CNY         1 non-null      float64       
 11  BRL         1 non-null      float64       
 12  ARS         1 non-null      float64       
 13  JPY         1 non-null      float64       
dtypes: datetime64[ns](2), float64(11), object(1)
memory usage: 240.0+ bytes


### Enviando datos a Redshift

In [ ]:
# Cargar credenciales desde el archivo YAML
from sqlalchemy import inspect
with open('credentials.yaml', 'r') as file:
    credenciales = yaml.safe_load(file)['redshift']

# Utilizar credenciales ocultas
username = credenciales['username']
password = credenciales['password']
host = credenciales['host']
port = credenciales['port']
database = credenciales['database']

# Construir la cadena de conexión
cadena_conexion = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Conectar al motor de la base de datos
engine = create_engine(cadena_conexion)
df_clean.to_sql('cambios_divisa', engine, if_exists='append', index=False)


1

### Leer los datos de redshift que fueron cargador previamente (al día actual) + los que ya se habían cargado (histórico)

In [ ]:
# Comprobar que los datos fueron cargados exitosamente en redshift al ser llamados
query = "SELECT * FROM cambios_divisa"
existing_data = pd.read_sql(query, engine)
existing_data

,start_date,end_date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy
0,2023-01-01,2023-01-01,MXN,0.047915,0.042389,0.051290,0.195107,0.000003,64.723687,4.244272,0.353795,0.271156,9.064450,6.714913
1,2023-01-02,2023-01-02,MXN,0.048115,0.042591,0.051374,0.195444,0.000003,65.250206,4.248940,0.354408,0.275563,9.152385,6.719676
2,2023-01-03,2023-01-03,MXN,0.048843,0.043039,0.051513,0.196521,0.000003,65.905399,4.264514,0.356214,0.282295,9.188812,6.761201
3,2023-01-04,2023-01-04,MXN,0.048615,0.042770,0.051572,0.196751,0.000003,65.569454,4.263326,0.355272,0.280121,9.209371,6.817845
4,2023-01-05,2023-01-05,MXN,0.049177,0.043443,0.051744,0.196822,0.000003,65.948080,4.276412,0.356129,0.276898,9.257505,6.898001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,2023-12-30,2023-12-30,MXN,0.053302,0.046251,0.058923,0.218004,0.000001,76.278052,4.905300,0.417095,0.286459,47.699017,8.309666
364,2023-12-31,2023-12-31,MXN,0.053293,0.046279,0.058913,0.217965,0.000001,76.264369,4.904421,0.417020,0.285874,47.767596,8.308381
365,2024-01-01,2024-01-01,MXN,0.053398,0.046307,0.058939,0.218260,0.000001,0.135853,4.903906,0.417207,0.286085,47.792660,8.318112
366,2024-01-02,2024-01-02,MXN,0.053654,0.046508,0.058705,0.218000,0.000001,76.835704,4.887596,0.416780,0.289061,47.583790,8.345262


### Puede haber duplicados en los datos previos, sin embargo, le pasamos un código para eliminarlos

In [ ]:
# Concatenar los datos y Eliminar duplicados
df_combined  = existing_data.drop_duplicates()
df_combined

,start_date,end_date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy
0,2023-01-01,2023-01-01,MXN,0.047915,0.042389,0.051290,0.195107,0.000003,64.723687,4.244272,0.353795,0.271156,9.064450,6.714913
1,2023-01-02,2023-01-02,MXN,0.048115,0.042591,0.051374,0.195444,0.000003,65.250206,4.248940,0.354408,0.275563,9.152385,6.719676
2,2023-01-03,2023-01-03,MXN,0.048843,0.043039,0.051513,0.196521,0.000003,65.905399,4.264514,0.356214,0.282295,9.188812,6.761201
3,2023-01-04,2023-01-04,MXN,0.048615,0.042770,0.051572,0.196751,0.000003,65.569454,4.263326,0.355272,0.280121,9.209371,6.817845
4,2023-01-05,2023-01-05,MXN,0.049177,0.043443,0.051744,0.196822,0.000003,65.948080,4.276412,0.356129,0.276898,9.257505,6.898001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,2023-12-29,2023-12-29,MXN,0.053302,0.046251,0.058923,0.217604,0.000001,76.278052,4.905300,0.417095,0.285926,47.537866,8.309666
363,2023-12-30,2023-12-30,MXN,0.053302,0.046251,0.058923,0.218004,0.000001,76.278052,4.905300,0.417095,0.286459,47.699017,8.309666
364,2023-12-31,2023-12-31,MXN,0.053293,0.046279,0.058913,0.217965,0.000001,76.264369,4.904421,0.417020,0.285874,47.767596,8.308381
365,2024-01-01,2024-01-01,MXN,0.053398,0.046307,0.058939,0.218260,0.000001,0.135853,4.903906,0.417207,0.286085,47.792660,8.318112


In [ ]:
# Cambiar tipo de datos
columnas = ['start_date', 'end_date']
df_combined[columnas] = df_combined[columnas].apply(pd.to_datetime, errors='coerce', axis=1)
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367 entries, 0 to 366
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_date  367 non-null    datetime64[ns]
 1   end_date    367 non-null    datetime64[ns]
 2   source      367 non-null    object        
 3   eur         367 non-null    float64       
 4   gbp         367 non-null    float64       
 5   usd         367 non-null    float64       
 6   pen         367 non-null    float64       
 7   btc         367 non-null    float64       
 8   krw         367 non-null    float64       
 9   inr         367 non-null    float64       
 10  cny         367 non-null    float64       
 11  brl         367 non-null    float64       
 12  ars         367 non-null    float64       
 13  jpy         367 non-null    float64       
dtypes: datetime64[ns](2), float64(11), object(1)
memory usage: 43.0+ KB


<ipython-input-122-864e68cf5c65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_combined[columnas] = df_combined[columnas].apply(pd.to_datetime, errors='coerce', axis=1)


### Volver a cargar datos pero ahora evitando duplicados

In [ ]:
# Escribir los datos combinados y sin duplicados de nuevo en la base de datos
df_combined.to_sql('cambios_divisa', engine, if_exists='replace', index=False)
df_combined

,start_date,end_date,source,eur,gbp,usd,pen,btc,krw,inr,cny,brl,ars,jpy
0,2023-01-01,2023-01-01,MXN,0.047915,0.042389,0.051290,0.195107,0.000003,64.723687,4.244272,0.353795,0.271156,9.064450,6.714913
1,2023-01-02,2023-01-02,MXN,0.048115,0.042591,0.051374,0.195444,0.000003,65.250206,4.248940,0.354408,0.275563,9.152385,6.719676
2,2023-01-03,2023-01-03,MXN,0.048843,0.043039,0.051513,0.196521,0.000003,65.905399,4.264514,0.356214,0.282295,9.188812,6.761201
3,2023-01-04,2023-01-04,MXN,0.048615,0.042770,0.051572,0.196751,0.000003,65.569454,4.263326,0.355272,0.280121,9.209371,6.817845
4,2023-01-05,2023-01-05,MXN,0.049177,0.043443,0.051744,0.196822,0.000003,65.948080,4.276412,0.356129,0.276898,9.257505,6.898001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362,2023-12-29,2023-12-29,MXN,0.053302,0.046251,0.058923,0.217604,0.000001,76.278052,4.905300,0.417095,0.285926,47.537866,8.309666
363,2023-12-30,2023-12-30,MXN,0.053302,0.046251,0.058923,0.218004,0.000001,76.278052,4.905300,0.417095,0.286459,47.699017,8.309666
364,2023-12-31,2023-12-31,MXN,0.053293,0.046279,0.058913,0.217965,0.000001,76.264369,4.904421,0.417020,0.285874,47.767596,8.308381
365,2024-01-01,2024-01-01,MXN,0.053398,0.046307,0.058939,0.218260,0.000001,0.135853,4.903906,0.417207,0.286085,47.792660,8.318112
